In [ ]:
import pandas as pd
import os
import datetime
from datetime import datetime, date, time, timezone, timedelta
import matplotlib.pyplot as plt
from opt_functions import SusRes_MBC

In [ ]:
Pidle_server_with_storage = 75 + 2*30 + 24*3 + 5
Pmax_server_with_storage = 75 + 2*30 + 36*7 + 24*8 + 18 #or 537 withought 2*30!!

Pidle_server = 75 + 2*30 + 24*3
Pmax_server = 75 + 2*30 + 36*7 + 24*8

# Alexnet

In [ ]:
#*********************************************************************
job_power_dir = '/Data_logs/outputs/Alexnet/alextnet_epoch0&1/power_logs_alexnet/'
#*********************************************************************

## reading gpu data

In [ ]:
job_power_data_gpu = pd.read_csv(job_power_dir + 'gpu_nvidia' +'.csv')
job_power_data_gpu.columns = [c.replace(' ', '_') for c in job_power_data_gpu.columns]
job_power_data_gpu.rename(columns={"_power.draw_[W]": "power", "_timestamp":"time", "_utilization.gpu_[%]":"gpu_util"
                              , "_temperature.gpu":"gpu_temp", "":""}, inplace=True)

job_power_data_gpu.drop(job_power_data_gpu.columns.difference(['index','power','time', 'gpu_util', 'gpu_temp']), 1, inplace=True)

job_power_data_gpu.power = job_power_data_gpu.power.str.replace('[^\d.]', '').astype(float)
job_power_data_gpu.gpu_util = job_power_data_gpu.gpu_util.str.replace('[^\d.]', '').astype(float)

job_power_data_gpu['time'] = job_power_data_gpu['time'].str.replace('\.\d\d\d', '')
job_power_data_gpu.time = pd.to_datetime(job_power_data_gpu.time, format='%Y-%m-%d %H:%M:%S')

In [ ]:
GPU0_data = job_power_data_gpu[job_power_data_gpu['index']==0].copy()
GPU1_data = job_power_data_gpu[job_power_data_gpu['index']==1].copy()
GPU2_data = job_power_data_gpu[job_power_data_gpu['index']==2].copy()
GPU3_data = job_power_data_gpu[job_power_data_gpu['index']==3].copy()
GPU4_data = job_power_data_gpu[job_power_data_gpu['index']==4].copy()
GPU5_data = job_power_data_gpu[job_power_data_gpu['index']==5].copy()
GPU6_data = job_power_data_gpu[job_power_data_gpu['index']==6].copy()
GPU7_data = job_power_data_gpu[job_power_data_gpu['index']==7].copy()

In [ ]:
GPU0_data = GPU0_data.set_index('time').resample('15s').mean()
GPU1_data = GPU1_data.set_index('time').resample('15s').mean()
GPU2_data = GPU2_data.set_index('time').resample('15s').mean()
GPU3_data = GPU3_data.set_index('time').resample('15s').mean()
GPU4_data = GPU4_data.set_index('time').resample('15s').mean()
GPU5_data = GPU5_data.set_index('time').resample('15s').mean()
GPU6_data = GPU6_data.set_index('time').resample('15s').mean()
GPU7_data = GPU7_data.set_index('time').resample('15s').mean()

## reading cpu data

In [ ]:
job_power_data_cpu_top = pd.read_csv(job_power_dir + 'cpu_top_fixed' +'.csv')

job_power_data_cpu_top.drop(job_power_data_cpu_top.columns.difference(['time','cpu_us', 'cpu_sy']), 1, inplace=True)

#********************************************************************
d1 = date(2021,11,18)
d2 = date(1900,1,1)
result1 = abs(d2-d1).days
#********************************************************************

job_power_data_cpu_top.time = pd.to_datetime(job_power_data_cpu_top.time, format='%H:%M:%S') + timedelta(result1) 
job_power_data_cpu_top = job_power_data_cpu_top.set_index('time').resample('15s').mean()

job_power_data_cpu_top['cpu_util'] = job_power_data_cpu_top['cpu_us'] + job_power_data_cpu_top['cpu_sy']

In [ ]:
#job_power_data_cpu_dstat = pd.read_csv(job_power_dir + 'cpu_dstat_fixed' +'.csv')

#job_power_data_cpu_dstat.drop(job_power_data_cpu_dstat.columns.difference(['system_time','cpu_usage_usr','cpu_usage_sys']), 1, inplace=True)
#job_power_data_cpu_dstat.rename(columns={"system_time": "time"}, inplace=True)

#job_power_data_cpu_dstat.time = pd.to_datetime(job_power_data_cpu_dstat.time, format='%Y-%m-%d %H:%M:%S')
#job_power_data_cpu_dstat = job_power_data_cpu_dstat.set_index('time').resample('3s').first()

## calculating average power

In [ ]:
power_df = job_power_data_cpu_top.copy() 
power_df.drop(columns=['cpu_us', 'cpu_sy'], inplace=True)

In [ ]:
power_df.cpu_util.plot()

In [ ]:
#****************************************************
power_df = power_df.iloc[50:-10]
#****************************************************
power_df.cpu_util.plot()

In [ ]:
date0 = power_df.index[0]
date1 = power_df.index[-1]

GPU0_data = GPU0_data.loc[date0:date1].copy()
GPU1_data = GPU1_data.loc[date0:date1].copy()
GPU2_data = GPU2_data.loc[date0:date1].copy()
GPU3_data = GPU3_data.loc[date0:date1].copy()
GPU4_data = GPU4_data.loc[date0:date1].copy()
GPU5_data = GPU5_data.loc[date0:date1].copy()
GPU6_data = GPU6_data.loc[date0:date1].copy()
GPU7_data = GPU7_data.loc[date0:date1].copy()

In [ ]:
power_df['server_power'] = Pidle_server_with_storage + (Pmax_server_with_storage-Pidle_server_with_storage)*(power_df['cpu_util']/100)
power_df['gpu_power'] = GPU0_data['power'] + GPU1_data['power'] +GPU2_data['power'] + GPU3_data['power'] + GPU4_data['power'] + GPU5_data['power'] + GPU6_data['power'] + GPU7_data['power']
power_df['gpu_util'] = (GPU0_data['gpu_util'] + GPU1_data['gpu_util'] +GPU2_data['gpu_util'] + GPU3_data['gpu_util'] + GPU4_data['gpu_util'] + GPU5_data['gpu_util'] + GPU6_data['gpu_util'] + GPU7_data['gpu_util'])/8
power_df['tot_power'] = power_df['server_power'] + power_df['gpu_power']